# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('../output_data/cities.csv')
vacation_df.count()

City          557
Lat           557
Lng           557
Max Temp      557
Humidity      557
Cloudiness    557
Wind Speed    557
Country       554
Date          557
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

humidity= vacation_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
better_vacation_df= vacation_df.loc[(vacation_df["Max Temp"]>70 ) & (vacation_df["Max Temp"]< 80) & (vacation_df["Wind Speed"]<= 10) 
                                    & (vacation_df["Cloudiness"]==0)].dropna()
better_vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,doha,25.2867,51.5333,77.68,81,0,4.61,QA,1637552105
114,salalah,17.0151,54.0924,75.29,46,0,5.75,OM,1637552166
124,grand gaube,-20.0064,57.6608,72.37,79,0,1.01,MU,1637552174
191,paysandu,-32.3214,-58.0756,73.42,33,0,7.23,UY,1637552198
418,zinder,13.8072,8.9881,74.62,21,0,7.90,NE,1637552403
440,lazaro cardenas,17.9583,-102.2000,76.24,84,0,2.86,MX,1637552420
495,santiago,-33.4569,-70.6483,72.86,44,0,6.91,CL,1637552262


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df=better_vacation_df
hotel_df["Hotel Name"]= ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,doha,25.2867,51.5333,77.68,81,0,4.61,QA,1637552105,
114,salalah,17.0151,54.0924,75.29,46,0,5.75,OM,1637552166,
124,grand gaube,-20.0064,57.6608,72.37,79,0,1.01,MU,1637552174,
191,paysandu,-32.3214,-58.0756,73.42,33,0,7.23,UY,1637552198,
418,zinder,13.8072,8.9881,74.62,21,0,7.90,NE,1637552403,
440,lazaro cardenas,17.9583,-102.2000,76.24,84,0,2.86,MX,1637552420,
495,santiago,-33.4569,-70.6483,72.86,44,0,6.91,CL,1637552262,


In [7]:
import pprint
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
#     "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
       
            # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

            # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"

        
#     try:
        # make request and print url
        response = requests.get(base_url, params=params).json()
#         pprint.pprint(response)
#         break

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
#     try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
41,doha,25.2867,51.5333,77.68,81,0,4.61,QA,1637552105,Movenpick Hotel Doha
114,salalah,17.0151,54.0924,75.29,46,0,5.75,OM,1637552166,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
124,grand gaube,-20.0064,57.6608,72.37,79,0,1.01,MU,1637552174,Veranda Paul et Virginie Hotel & Spa
191,paysandu,-32.3214,-58.0756,73.42,33,0,7.23,UY,1637552198,Gran Hotel Paysandú
418,zinder,13.8072,8.9881,74.62,21,0,7.90,NE,1637552403,Hotel de Ville
440,lazaro cardenas,17.9583,-102.2000,76.24,84,0,2.86,MX,1637552420,quinta antigua
495,santiago,-33.4569,-70.6483,72.86,44,0,6.91,CL,1637552262,Che Lagarto Hostel Santiago


In [8]:
# for index, row in hotel_df.iterrows():
#     print(row)
#     break


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# hotel_info

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))